# 06. Vision Transformets & Low-rank Adapters

Трансфореры могут применяться не только к тексту, но и картинкам! Впервые Vision Transformer был предложен в статье [AN IMAGE IS WORTH 16X16 WORDS:
TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE](https://arxiv.org/pdf/2010.11929)

![](https://www.kaggleusercontent.com/kf/48380283/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..HE7ZJfbhaOVh_1DkzuBT9A.CNUBUXLLZa-MiSr_1ibipk2ZMiI_N1g4bO191I9PXphP_6BbS5hjqm_Wqy0dCaB9WfESQwQlXnLmyV5Jv98YsrfIhqyhFlFHjhmzOFHmclaue7p_bN4D-q4Eg34R4VpIrJ7CXfSle6_ZX_S5yNnG2fdxdrFPGX2fTHp_gZ1EK1hhN0gqfiXJnQQDfO4ADggv285-wu47EnJVvTQ5N5oQ8VO4hXtZ65dEXdKCy85AXMDC2321MMuuaQ4sU9UBtUYzUoN1EJGcv50Nz2F6Ctmu6G8QeER8rLLSdqv4yCVGVy7G4js5eZxrzCk7DusX8cMrShuchgQSEP_3Gze2568fG2kz8_vXXrMxIPpsJoeQMFgQLWe5Y6wKISoZVYRahB0clveLNMDvgLK52dnr01DL207Rpt9Wo7yJLuHyOKURy4r80XLE30QyqXQcbBqSaYZabK_RIyyeEsMIwnJxGCITLy0nGGGUbGBr90ZDOS4_71VOJOA5HdAD3BfAxRpKdSJdi3FgwuX6B19VLKijlpdW-UUYd2o-9TxaLbYdlNMP8sdykbeq_gI5K9NnCgUbtA0VIuJH3BOw-eRi_tXB_M9qmBAtdb4SfRyNOATrjvoOxFAaHXBOc3esj5Dk6VwogPxljs-dT8-ExK3SQw_c-kJ90O32rcnWts-ua7y5rckADP8.rf23XG5vBQOKoMTFq6CU2w/vit-arch.png)


**(1)** Используется только часть преобразователя Encoder

**(2)** Изображение разбивается на фрагменты фиксированного размера. Таким образом, один из этих фрагментов (aka patches) может иметь размер 16x16 или 32x32, как предлагается в статье. Больше фрагментов означает, что проще обучать сеть, поскольку сами фрагменты становятся меньше.

**(3)** Затем фрагменты разворачиваются (сглаживаются) и отправляются для дальнейшей обработки в сеть.

**(4)** Модель не имеет никакого представления о положении образцов в последовательности, так как каждый сэмпл является фрагментом из входного изображения. Поэтому изображение подается вместе с позиционным вектором. Здесь следует отметить, что позиционные эмбеддинги также обучаемы, поэтому  фактически не нужно передавать жестко закодированные векторы относительно их позиций.

**(5)** Также есть специальный токен в начале, как и в BERT.

**(6)** Каждый фрагмент изображения сначала разворачивается (сглаживается) в большой вектор и умножается на  эмбеддинг, который также обучаем, создавая эмбеддинги фрагментов. Патчи объединяются с позиционным вектором эмбеддингов, и он подается в трансформер.

**(7)** Единственное отличие в том, что вместо декодера вывод из кодера передается непосредственно в FFNN для получения вывода классификации.

[Источник картинки](https://www.kaggle.com/code/abhinand05/vision-transformer-vit-tutorial-baseline)

### 1) Load transformers packages & dataset

`transformers` - библиотека, ассоциируемая с HuggingFace, используется для высокоуровневой работы с трансформерами

`accelerate` - ускоряет обучение и инференс моделей

`evaluate` - библиотека для методов оценки перформанса моделей, например, подсчет точности

`datasets` - библиотека, ассоциируемая с HuggingFace, используется для высокоуровневой работы с датасетами

`peft` - библиотека с разнообразными адаптерами, например, LoRA

In [ ]:
!pip install transformers accelerate evaluate datasets git+https://github.com/huggingface/peft -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification
import torch
from datasets import load_from_disk

Добавим несколько преобразований данных

Датасет food-101 можно скачать, например, [здесь](https://www.kaggle.com/datasets/dansbecker/food-101)

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

# Target dataset
dataset = load_from_disk("food-101_train")

# Data prepapator for a model
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

# Extract parameters from image_processor
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

train_transforms = Compose(
    [
        RandomResizedCrop(image_processor.size["height"]),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(image_processor.size["height"]),
        CenterCrop(image_processor.size["height"]),
        ToTensor(),
        normalize,
    ]
)

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Переводим лейблы из строк в числа

In [ ]:
label2id, id2label = dict(), dict()
labels = dataset.features["label"].names
# Go through the labels and save corresponding indexes
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

Обучающая и валидационная выборки

In [ ]:
# split up training into training + validation
splits = dataset.train_test_split(test_size=0.1)
train_ds = splits["train"]
val_ds = splits["test"]

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

### 2) Model loading

Загрузим модель

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer


model = AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
print_trainable_parameters(model)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([101]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([101, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 85876325 || all params: 85876325 || trainable%: 100.00


### 3) Low-rank adaptation

[LoRA](https://arxiv.org/pdf/2106.09685) -— известный метод до-обучения трансформаторов. Можно **разложить** весовую матрицу трансформатора на две меньшие по размерности матрицы и обучать только их.

peft позволяет делать это высокоуровнено, без погружения в детали реализации

In [ ]:
from peft import LoraConfig, get_peft_model
# Load config
config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)


trainable params: 1257317 || all params: 87133642 || trainable%: 1.44


Адаптер готов. Обратите внимание на обучаемый процент параметров

### 4) Training of transformer

In [ ]:
from transformers import TrainingArguments, Trainer


batch_size = 128

args = TrainingArguments(
    f"fine-tunned-model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    label_names=["labels"],
    report_to="none"
)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
trainer = Trainer(
    lora_model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)
train_results = trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
0,2.252800,0.885445,0.758575
2,0.679500,0.772291,0.792876
4,0.501900,0.756952,0.791557


### 5) Save adapter

Вместо того, чтобы сохранять весь ViT, сохраним только адаптер для дальнейшей загрузки и использования

In [ ]:
from peft import PeftModel
trainer.model.save_pretrained("my_adapter")

In [ ]:
finetuned_model = PeftModel.from_pretrained(model,
                                  "my_adapter",
                                  torch_dtype=torch.float16,
                                  is_trainable=False,
                                  device_map="auto"
                                  )
finetuned_model = finetuned_model.merge_and_unload()

In [ ]:
for i, data in enumerate(val_ds):
  data = collate_fn([data])
  image = data['pixel_values'].to('cuda')

  outputs = finetuned_model(image)
  predicted_class_idx = outputs.logits.argmax(-1).item()
  print(id2label[predicted_class_idx], id2label[data['labels'].item()])
  if i > 15:
    break